Date: 0512
### Run Treeshrink

In [62]:
import subprocess

def run_treeshrink(tree_file):
    cmd = ["run_treeshrink", "--tree", tree_file]
    subprocess.run(cmd, capture_output=True, text=True)

tree_file = "/home/tim/project/GTDB_TREE/data/r220/raw_name/bac120_r220_dropped.tree"
run_treeshrink(tree_file)

ERROR: Error in parse(text = x, srcfile = src): <text>:1:8: unexpected symbol
1: import subprocess
           ^


### Check shrinked taxa

In [1]:
library(dplyr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
shrink_file <- "/home/tim/project/GTDB_TREE/data/r220/raw_name/bac120_r220_dropped_treeshrink/output.txt"
taxa_table <- "/home/tim/project/GTDB_TREE/data/r220/raw_name/rep_bac120_taxonomy_r220.tsv"
taxa_df <- read.delim(taxa_table, stringsAsFactors = FALSE, sep = "\t")
shrinked_taxa <- readLines(shrink_file)
shrinked_taxa <- strsplit(shrinked_taxa, "\t")[[1]]
length(shrinked_taxa)

[1] 35

In [6]:
# Summarize data by phylum, sort by descending number of species
phylum_summary <- taxa_df %>%
    group_by(Phylum) %>%
    summarise(
        n_species = n(),
        .groups = "drop"
    ) %>%
    arrange(desc(n_species))

In [7]:
shrinked_df <- taxa_df %>% filter(ID %in% shrinked_taxa)
shrink_summary <- shrinked_df %>% group_by(Phylum) %>% summarise(n_drop = n())
shrink_summary

Phylum,n_drop
<chr>,<int>
p__Bacillota_I,4
p__Bacteroidota,8
p__Chloroflexota,3
p__Patescibacteria,14
p__Pseudomonadota,6


In [8]:
left_join(shrink_summary,phylum_summary,"Phylum")

Phylum,n_drop,n_species
<chr>,<int>,<int>
p__Bacillota_I,4,2903
p__Bacteroidota,8,14787
p__Chloroflexota,3,2749
p__Patescibacteria,14,4581
p__Pseudomonadota,6,27962


### Generate new taxa_df

In [48]:
taxa_df <- taxa_df %>% filter(!(ID %in% shrinked_taxa))
taxa_table_shrinked <- "/home/tim/project/GTDB_TREE/data/r220/raw_name/rep_bac120_taxonomy_r220_shrinked.tsv"
write.table(taxa_df,file = taxa_table_shrinked,sep = "\t",quote = FALSE, row.names = FALSE) 

### Shorten taxa name

In [49]:
taxa_df$ID <- sub(".*_(.*)\\..*", "G\\1", taxa_df$ID)

In [50]:
taxa_df %>% count(ID) %>% count(n)

Storing counts in `nn`, as `n` already present in input
ℹ Use `name = "new_name"` to pick a new name.


n,nn
<int>,<int>
1,107186


In [57]:
taxa_table_new <- "/home/tim/project/GTDB_TREE/data/r220/rep_bac120_taxonomy.tsv"
write.table(taxa_df,file = taxa_table_new,sep = "\t",quote = FALSE, row.names = FALSE) 

In [58]:
library(ape)
old_tree <- read.tree("/home/tim/project/GTDB_TREE/data/r220/raw_name/bac120_r220_dropped.tree")

In [59]:
new_tree <- drop.tip(old_tree, shrinked_taxa)

In [60]:
new_tree$tip.label <- sub(".*_(.*)\\..*", "G\\1", new_tree$tip.label)

In [61]:
species_in_table <- taxa_df %>% pull(ID)
species_in_tree <- new_tree$tip.label
setequal(species_in_table, species_in_tree)

[1] FALSE

In [62]:
length(setdiff(species_in_table, species_in_tree))

[1] 0

In [63]:
setdiff(species_in_table, species_in_tree)
setdiff(species_in_tree, species_in_table)

character(0)

[1] "G026989995"

In [65]:
new_tree <- drop.tip(new_tree, "G026989995")

In [66]:
write.tree(new_tree, "/home/tim/project/GTDB_TREE/data/r220/bac120_r220.tree")

In [67]:
writeLines(new_tree$tip.label, "/home/tim/project/GTDB_TREE/data/r220/taxa_list.txt")

### Rename taxa for alignment files

In [1]:
alignment_dir = "/home/tim/project/GTDB_TREE/alignment/raw_r220/individual"
output_dir = "/home/tim/project/GTDB_TREE/alignment/r220/individual"

In [2]:
import os
import re

def modify_taxa_name(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        if filename.endswith('.fna') or filename.endswith('.faa') or filename.endswith('.fasta'):
            with open(os.path.join(input_folder, filename), 'r') as infile, \
                 open(os.path.join(output_folder, filename), 'w') as outfile:
                for line in infile:
                    if line.startswith('>'):
                        line = re.sub(r".*_(.*)\..*", r">G\1", line)
                    outfile.write(line)


In [3]:
# modify_taxa_name(alignment_dir, output_dir)

### Prune alignment

In [19]:
import sys
from pathlib import Path
sys.path.append('./small_func')
from sample_alignment import *

loci_dir = Path("/home/tim/project/GTDB_TREE/alignment/r220/individual")
taxa_list = Path("/home/tim/project/GTDB_TREE/data/r220/taxa_list.txt")
output_folder = Path("/home/tim/project/GTDB_TREE/alignment/r220/pruned_alignments")

In [20]:
sample_alignment(loci_dir, taxa_list, output_folder)

Sampling alignments...
Input a single taxa file: /home/tim/project/GTDB_TREE/data/r220/taxa_list.txt. Sampling sequences for 120 loci.
Number of input species: 107186
Remaining 120 alignments. Deleted 0 alignments.


### Spilt training and testing loci

In [24]:
import os
import shutil
import random

def split_files(src_dir, train_num):
    files = os.listdir(src_dir)
    random.shuffle(files)

    train_dir = os.path.join(os.path.dirname(src_dir), 'train')
    test_dir = os.path.join(os.path.dirname(src_dir), 'test')
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)
    
    for i, file in enumerate(files):
        src_file = os.path.join(src_dir, file)
        if i < train_num:
            dst_file = os.path.join(train_dir, file)
        else:
            dst_file = os.path.join(test_dir, file)
        shutil.move(src_file, dst_file)
    shutil.rmtree(src_dir)


In [25]:
split_files(output_folder, 100)

FileNotFoundError: [Errno 2] No such file or directory: '/home/tim/project/GTDB_TREE/alignment/r220/pruned_alignments'

### Generate integrity check table

In [ ]:
import subprocess
import os
from quality_trimming import generate_combined_df

train_loc_path = "/home/tim/project/GTDB_TREE/alignment/r220/train"
test_loc_path = "/home/tim/project/GTDB_TREE/alignment/r220/test"
taxa_file = "/home/tim/project/GTDB_TREE/data/r220/rep_bac120_taxonomy.tsv"
output_dir = "../data/r220"

# for test
# train_loc_path = "/mnt/data/dayhoff/home/u7457359/project/GTDB/GTDB_TREE/Result/p__Actinobacteriota_200/p__Actinobacteriota_200_002/loci/training_loci"
# test_loc_path = "/mnt/data/dayhoff/home/u7457359/project/GTDB/GTDB_TREE/Result/p__Actinobacteriota_200/p__Actinobacteriota_200_002/loci/testing_loci"

generate_combined_df(train_loc_path, test_loc_path, taxa_file, output_dir)